In [ ]:
# Define qual implementação de MapReduce usar: 'java' ou 'python'
MAPREDUCE_LANG = 'java' # Altere para 'python' para usar a implementação em Python

print(f"Usando implementação de MapReduce: {MAPREDUCE_LANG.upper()}")

# Lab 3: Pipeline Interativo de Cálculo de Valor por Perfil de Pet

Este notebook executa o pipeline de cálculo de Valor por Perfil de Pet (LTV). As células abaixo irão compilar o código MapReduce em Java, ingerir dados do PostgreSQL com Sqoop, executar o job no Hadoop e verificar os resultados.

## 1. Compilando a Aplicação MapReduce

A primeira etapa é compilar nosso código-fonte Java em um arquivo `.jar` executável. A célula abaixo usa o Maven para isso. Ela navega até o diretório do projeto e executa o `mvn package`. A flag `-q` é para uma saída mais limpa.

In [ ]:
%%bash
if [ "$MAPREDUCE_LANG" == "java" ]; then
    cd ltv-by-pet-profile
    mvn package -q
    echo "JAR compilado com sucesso em: target/ltv-by-pet-profile-1.0-SNAPSHOT.jar"
fi

## 2. Origem dos dados

Execute a célula abaixo para se conectar novamente e fazer uma consulta `SELECT` com os dados inseridos anteriormente.

In [ ]:
import psycopg2
import os

# As credenciais e o host são baseados no arquivo docker-compose.txt
DB_HOST = "localhost" # Nome do serviço no Docker Compose
DB_NAME = "postgres"
DB_USER = "postgres"
DB_USER_PWD = "postgres"

try:
    conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_USER_PWD)
    cur = conn.cursor()
    
    cur.execute("SELECT CONCAT(p.species, ';', p.fur_type) AS perfil_pet, (hc.quantity * pr.price) AS valor_compra FROM  purchase hc JOIN product pr ON hc.product_id = pr.product_id JOIN pet p ON hc.tutor_id = p.tutor_id")
    rows = cur.fetchall()
    
    print("Agendamentos encontrados:")
    for row in rows:
        print(row)
        
except Exception as e:
    print(f"Ocorreu um erro: {e}")
finally:
    if 'conn' in locals() and conn is not None:
        cur.close()
        conn.close()

## 3. Ingestão de Dados com Sqoop

Agora, vamos importar os dados de compras do PostgreSQL para o HDFS. O comando `hdfs dfs -rm` é usado para remover o diretório de destino antes da importação, garantindo que possamos executar esta célula várias vezes sem erros.

A query Sqoop já realiza um JOIN para obter o perfil do pet e o valor da compra, simplificando o MapReduce posterior.

In [ ]:
%%bash
INPUT_DIR=/petshop/input_ltv

# Remove o diretório de entrada se ele já existir
hdfs dfs -test -d $INPUT_DIR
if [ $? -eq 0 ]; then
    echo "Removendo diretório HDFS existente: $INPUT_DIR"
    hdfs dfs -rm -r $INPUT_DIR
fi

echo "Iniciando importação com Sqoop..."
sqoop import \
    --connect jdbc:postgresql://localhost:5432/postgres \
    --username postgres \
    --password postgres \
    --query "SELECT \
                CONCAT(p.species, ';', p.fur_type) AS perfil_pet, \
                (hc.quantity * pr.price) AS valor_compra \
            FROM  purchase hc \
            JOIN product pr ON hc.product_id = pr.product_id \
            JOIN pet p ON hc.tutor_id = p.tutor_id \
            WHERE \$CONDITIONS" \
    --target-dir $INPUT_DIR \
    --m 1 \
    --split-by hc.purchase_id

echo "\nImportação concluída. Verificando os dados no HDFS:"
hdfs dfs -ls $INPUT_DIR
hdfs dfs -cat $INPUT_DIR/part-m-00000 | head -n 5

## 4. Execução do Job MapReduce

Com os dados no HDFS, podemos executar nosso job MapReduce. A célula abaixo submete o `.jar` que compilamos para o Hadoop. O resultado será salvo no diretório `/petshop/output_ltv`.

In [ ]:
%%bash
INPUT_DIR=/petshop/input_ltv
OUTPUT_DIR=/petshop/output_ltv

# Remove o diretório de saída se ele já existir
hdfs dfs -test -d $OUTPUT_DIR
if [ $? -eq 0 ]; then
    echo "Removendo diretório HDFS de saída existente: $OUTPUT_DIR"
    hdfs dfs -rm -r $OUTPUT_DIR
fi

if [ "$MAPREDUCE_LANG" == "java" ]; then
    JAR_PATH=/home/jovyan/work/labs/mapreduce-jobs/ltv-by-pet-profile/target/ltv-by-pet-profile-1.0-SNAPSHOT.jar
   
    echo "Executando o job MapReduce (Java)...
    hadoop jar $JAR_PATH $INPUT_DIR $OUTPUT_DIR

elif [ "$MAPREDUCE_LANG" == "python" ]; then
    MAPPER_PATH=labs/mapreduce-jobs/ltv-by-pet-profile-python/mapper.py
    REDUCER_PATH=labs/mapreduce-jobs/ltv-by-pet-profile-python/reducer.py
 
    echo "Executando o job MapReduce (Python)...
    hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar \
        -file $MAPPER_PATH -mapper $MAPPER_PATH \
        -file $REDUCER_PATH -reducer $REDUCER_PATH \
        -input $INPUT_DIR \
        -output $OUTPUT_DIR
fi

echo "\nJob concluído!"

## 5. Verificação do Resultado

Vamos verificar o resultado processado no HDFS. O arquivo `part-r-00000` deve conter o `perfil_pet` e o `valor_total`.

In [ ]:
%%bash
OUTPUT_DIR=/petshop/output_ltv
echo "Conteúdo do diretório de saída:"
hdfs dfs -ls $OUTPUT_DIR

echo "\nResultado do processamento:"
hdfs dfs -cat $OUTPUT_DIR/part-r-00000